```shell
conda create -n AI python=3.9; conda activate AI
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple transformers==4.39.2
conda install cudatoolkit==11.8
pip3 install numpy --pre torch torchvision torchaudio --force-reinstall --index-url https://pypi.tuna.tsinghua.edu.cn/simple
pip install auto-gptq --no-build-isolation
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple ipykernel evaluate tqdm scikit-learn
````

In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
from pprint import pprint
from peft import get_peft_model, LoraConfig, PeftModel
import evaluate
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [2]:
# dataset = load_dataset("stanfordnlp/sst2")

### 一、加载语句分类模型

In [3]:
model_checkpoint = "FacebookAI/roberta-large"
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

# 这样会自动创建一个分类模型
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)
model

/home/ljr/miniconda3/envs/AI/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

### 二、创建little-SST-2数据集

In [22]:
def small_sst(path): 
    f = open(path)
    ds = []
    label = []
    for line in f:
        text, l = line.split('\t')
        ds.append(text)
        label.append(int(l[0]))
    return {'text': ds, 'label': label}

train_ds = Dataset.from_dict(small_sst("SST-2/train.txt"))
test_ds = Dataset.from_dict(small_sst("SST-2/test.txt"))
# 创建DatasetDict对象
dataset = DatasetDict({
    'train': train_ds,
    'validation': test_ds
})

# 打印数据集信息
pprint(dataset)

{'train': Dataset({
    features: ['text', 'label'],
    num_rows: 7393
}),
 'validation': Dataset({
    features: ['text', 'label'],
    num_rows: 1749
})}


In [5]:
# 创建分词器
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [6]:
# create tokenize function
def tokenize_function(examples):
    text = examples['text']

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )
    return tokenized_inputs

# 数据预处理
tokenized_dataset = dataset.map(tokenize_function, batched=True)
pprint(tokenized_dataset)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/7393 [00:00<?, ? examples/s]

Map:   0%|          | 0/1749 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 7393
}),
 'validation': Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1749
})}


In [8]:
# 创建评估函数
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [9]:
# 定义一些栗子， 这里先测试没有经过微调的模型
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", 
             "This is not worth watching even once.", "I really love it.", 
             "I cannot stand it.", 
             "They didn't praise him slightly."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt")
    logits = model(inputs).logits
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Positive
Not a fan, don't recommed. - Positive
Better than the first one. - Positive
This is not worth watching even once. - Positive
I really love it. - Positive
I cannot stand it. - Positive
They didn't praise him slightly. - Positive


### 三、用Lora微调模型

In [10]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['query', 'key', 'value']) # 这个是可以微调的层

In [11]:
# 打印参数情况
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,641,474 || all params: 357,003,268 || trainable%: 0.4598


In [15]:
# 超参数
lr = 1e-5
batch_size = 8
num_epochs = 7

In [16]:
# 训练的信息
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
# 创建trainer对象
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# 训练模型
trainer.train()

/home/ljr/miniconda3/envs/AI/lib/python3.9/site-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/6475 [00:00<?, ?it/s]

{'loss': 0.6949, 'grad_norm': 2.287585496902466, 'learning_rate': 9.227799227799229e-06, 'epoch': 0.54}


  0%|          | 0/219 [00:00<?, ?it/s]

{'eval_loss': 0.27364063262939453, 'eval_accuracy': {'accuracy': 0.9342481417953116}, 'eval_runtime': 8.7085, 'eval_samples_per_second': 200.839, 'eval_steps_per_second': 25.148, 'epoch': 1.0}


...

  0%|          | 0/219 [00:00<?, ?it/s]

{'eval_loss': 0.2342844307422638, 'eval_accuracy': {'accuracy': 0.9536878216123499}, 'eval_runtime': 8.8182, 'eval_samples_per_second': 198.339, 'eval_steps_per_second': 24.835, 'epoch': 7.0}


/home/ljr/miniconda3/envs/AI/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 657.2996, 'train_samples_per_second': 78.733, 'train_steps_per_second': 9.851, 'train_loss': 0.329482901436942, 'epoch': 7.0}


TrainOutput(global_step=6475, training_loss=0.329482901436942, metrics={'train_runtime': 657.2996, 'train_samples_per_second': 78.733, 'train_steps_per_second': 9.851, 'train_loss': 0.329482901436942, 'epoch': 7.0})

In [18]:
# 使用微调后的模型
model.to('cpu') 

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu") # moving to mps for Mac (can alternatively do 'cpu')

    logits = model(inputs).logits
    print(logits)
    predictions = torch.max(logits, 1).indices
    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
tensor([[-3.0831,  3.8758]], grad_fn=<AddmmBackward0>)
It was good. - Positive
tensor([[ 3.4822, -3.7452]], grad_fn=<AddmmBackward0>)
Not a fan, don't recommed. - Negative
tensor([[-1.9666,  2.8543]], grad_fn=<AddmmBackward0>)
Better than the first one. - Positive
tensor([[ 3.4910, -3.7618]], grad_fn=<AddmmBackward0>)
This is not worth watching even once. - Negative
tensor([[-3.2008,  3.4340]], grad_fn=<AddmmBackward0>)
I really love it. - Positive
tensor([[ 3.5214, -3.6994]], grad_fn=<AddmmBackward0>)
I cannot stand it. - Negative
tensor([[ 1.6578, -1.9693]], grad_fn=<AddmmBackward0>)
They didn't praise him slightly. - Negative


In [19]:
#  保存模型
path = "pretrained_q_k_v"
model = model.save_pretrained(path)
tokenizer = tokenizer.save_pretrained(path)

/home/ljr/miniconda3/envs/AI/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### 四、加载模型，进行推理

In [20]:
base_model = "FacebookAI/roberta-large"
model_checkpoint = "pretrained_q_k_v"
# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

inference_model = AutoModelForSequenceClassification.from_pretrained(
    base_model, num_labels=2, id2label=id2label, label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = PeftModel.from_pretrained(inference_model, model_checkpoint)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
text = "I like it."
# 对输入文本进行编码
inputs = tokenizer(text, return_tensors="pt")

# 模型推理
with torch.no_grad():
    outputs = model(**inputs)

# 获取预测结果
predictions = torch.argmax(outputs.logits, dim=-1)

# 打印预测结果
print(outputs)
print(f"Predicted label: {id2label[predictions.item()]}")

SequenceClassifierOutput(loss=None, logits=tensor([[-2.8761,  3.6284]]), hidden_states=None, attentions=None)
Predicted label: Positive
